In [1]:
import pandas as pd
import numpy as np
import matplotlib
import warnings 
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
# Dealing with datetime features
from datetime import *

# Timer
import time

## Lets understand the data here . 

#### The values obtained from the textmining and Kmeans clustering is merged into the main dataset for Train data and Test data and will be used as analysis . 


#### The grievance description column is removed and the Labels column derived from clustering using the pdf and the doc2 vec is used instead for further analysis . 

In [2]:
data = pd.read_csv("Grievance_data_with_label.csv", sep=",",header='infer', )
new_data = pd.read_csv("Grievance_testdata_with_label.csv", sep=",",header='infer')

In [3]:
#This function computes the percentage of missing data per header name for a sub-frame of the CCD.
def missing_number(dataframe, header_name):
    count = 0
    for x in dataframe[header_name]:
        try:
            if np.isnan(x):
                count+=1
        except TypeError:
            pass
    return (count)



#This function counts unique elements in columns
def count_unique(dataframe, header_name):
    z = dataframe[header_name].unique()
    return sum(1 for v in z if pd.notnull(v))

In [4]:
r = []
for col in data.columns:
    r.append([col, missing_number(data,col),count_unique(data,col)])
missing_data = pd.DataFrame(r, columns=['Column Name', 'Missing Val', 'Unique'])
print(missing_data)

             Column Name  Missing Val  Unique
0            GrievanceID            0   53680
1                 BankID            0      86
2                  State          121      59
3        DateOfGrievance            0    1222
4     Grievance_Category            0      67
5   GrievanceDescription            0   46662
6         LineOfBusiness            0       4
7     ResolutionComments            0       4
8               Disputed            0       2
9       DateOfResolution            0    1248
10             BankGrade            0       3
11                Labels            0       2


In [5]:
r = []
for col in new_data.columns:
    r.append([col, missing_number(new_data,col),count_unique(new_data,col)])
missing_data = pd.DataFrame(r, columns=['Column Name', 'Missing Val', 'Unique'])
print(missing_data)

             Column Name  Missing Val  Unique
0            GrievanceID            0   27954
1                 BankID            0      22
2                  State           75      61
3        DateOfGrievance            0    1168
4     Grievance_Category            0      59
5   GrievanceDescription            0   24137
6         LineOfBusiness            0       4
7     ResolutionComments            0       4
8               Disputed            0       2
9       DateOfResolution            0    1198
10                Labels            0       2


In [6]:
for column in data.columns:
    data[column].fillna(data[column].mode()[0], inplace=True)

In [7]:
for column in new_data.columns:
    new_data[column].fillna(new_data[column].mode()[0], inplace=True)

### temporarily creating a copy of the dataset

## Lets visualize the categories in test file

In [ ]:
fig=plt.figure(figsize=(18, 40), dpi= 80, facecolor='w', edgecolor='k')
count_plot = sns.countplot(y = new_df['Grievance_Category'],order=new_df['Grievance_Category'].value_counts().index)
count_plot.set_ylabel('Grievance_Category')
count_plot.set_title('No. of Grievances in each Sector')
plt.xlabel('Num_of_Grievances')


In [8]:
## Only interested in subset of data without consumer complaints
##Taking subset of the data

df = data[['GrievanceID','BankID','State','DateOfGrievance','Grievance_Category','LineOfBusiness','ResolutionComments','Disputed','DateOfResolution','BankGrade','Labels']]

In [9]:
## Only interested in subset of data without consumer complaints
##Taking subset of the data

new_df = new_data[['GrievanceID','BankID','State','DateOfGrievance','Grievance_Category','LineOfBusiness','ResolutionComments','Disputed','DateOfResolution','Labels']]
#new_df = data_test_encoded[['GrievanceID','BankID','State','DateOfGrievance','Grievance_Category','LineOfBusiness','ResolutionComments','Disputed','DateOfResolution','Labels']]

In [ ]:
df

### Feature Engineering is carried out on both Train data and test data 

In [10]:
#Type casting
df['DateOfGrievance'] = pd.to_datetime(df['DateOfGrievance'], format='%m/%d/%Y')
df['DateOfResolution'] = pd.to_datetime(df['DateOfResolution'], format='%m/%d/%Y')

In [11]:
#Type casting
new_df['DateOfGrievance'] = pd.to_datetime(new_df['DateOfGrievance'], format='%m/%d/%Y')
new_df['DateOfResolution'] = pd.to_datetime(new_df['DateOfResolution'], format='%m/%d/%Y')

In [12]:
from datetime import timedelta

df['Processing_time'] = df['DateOfResolution']-df['DateOfGrievance']
df.loc[df['Processing_time']<timedelta(days=0),'DateOfResolution'] = df['DateOfGrievance']

In [13]:
from datetime import timedelta

new_df['Processing_time'] = new_df['DateOfResolution']-new_df['DateOfGrievance']
new_df.loc[new_df['Processing_time']<timedelta(days=0),'DateOfResolution'] = new_df['DateOfGrievance']

In [14]:
df['Year'] = pd.to_datetime(df['DateOfGrievance'], format = '%d%b%Y')
df['Year']= df['Year'].dt.year

In [15]:
new_df['Year'] = pd.to_datetime(new_df['DateOfGrievance'], format = '%d%b%Y')
new_df['Year']= new_df['Year'].dt.year

In [16]:
df['Processing_time'] = df['Processing_time'].astype('timedelta64[D]').astype(int)

In [17]:
new_df['Processing_time'] = new_df['Processing_time'].astype('timedelta64[D]').astype(int)

### Y variable classification check

In [18]:
label = pd.DataFrame(df.BankGrade.value_counts())
label.columns = ['BankGrade_cnt']
label['Bankrating'] = label.BankGrade_cnt/df.shape[0]

base = 0.972434

label

BankGrade_cnt  Bankrating
satisfactory          34881    0.649795
deficient             14304    0.266468
outstanding            4495    0.083737

In [19]:
df.BankGrade[df.BankGrade == 'deficient'] = 0
df.BankGrade[df.BankGrade == 'satisfactory'] = 1
df.BankGrade[df.BankGrade == 'outstanding'] = 2

df.BankGrade = df.BankGrade.astype('int')

In [20]:
def freq_by_cat(data, cat):
    '''
    Input: data is the whole data set;
           cat is the categorical feature you want to calculate the frequency of each category inside this feature.
    Output: a dataframe
    '''
    
    cat = str(cat)
    # Calculate the frequency of each cat
    freq = data.groupby('%s'%cat).count()/53680
    # reset index
    freq['%s'%cat] = freq.index
    freq.columns = ['freq_by_%s'%cat, '%s'%cat]
    freq = freq.reset_index(drop=True)
    
    return freq

In [21]:
Issue = df[['Grievance_Category', 'BankGrade']]
freq_by_issue = freq_by_cat(Issue, 'Grievance_Category')
#freq_by_issue

# One new feature: freq_by_issue
df = pd.merge(df, freq_by_issue, on='Grievance_Category')

In [22]:
Business = df[['LineOfBusiness', 'BankGrade']]

freq_by_business = freq_by_cat(Business, 'LineOfBusiness')

# One new feature: freq_by_product
df = pd.merge(df, freq_by_business, on='LineOfBusiness')

In [23]:
State = df[['State', 'BankGrade']]

freq_by_State = freq_by_cat(State, 'State')

# One new feature: freq_by_state
df = pd.merge(df, freq_by_State, on='State')

In [24]:
df = df.drop(['GrievanceID','BankID','DateOfGrievance','DateOfResolution','Year'],axis =1)
new_df = new_df.drop(['GrievanceID','BankID','DateOfGrievance','DateOfResolution','Year'],axis =1)

In [26]:
df.columns.tolist() == new_df.columns.tolist()

False

### Next we will concatenate the train and test data to have equal attributes for running the ML models 

In [27]:
train_objs_num = len(df)

In [28]:
dataset = pd.concat(objs=[df, new_df], axis=0)

In [29]:
dataset = pd.get_dummies(dataset,sparse= True)

In [30]:
import copy
train_df = copy.copy(dataset[:train_objs_num])

In [31]:
test_df = copy.copy(dataset[train_objs_num:])

In [32]:
test_df.head()

BankGrade  Labels  Processing_time  freq_by_Grievance_Category  \
0        NaN       0               11                         NaN   
1        NaN       0               15                         NaN   
2        NaN       1               11                         NaN   
3        NaN       1               11                         NaN   
4        NaN       0               11                         NaN   

   freq_by_LineOfBusiness  freq_by_State  Disputed_No  Disputed_Yes  \
0                     NaN            NaN            1             0   
1                     NaN            NaN            1             0   
2                     NaN            NaN            1             0   
3                     NaN            NaN            1             0   
4                     NaN            NaN            1             0   

   Grievance_Category_APR or interest rate  \
0                                        0   
1                                        0   
2                                        0   
3                                        0   
4                                        0   

   Grievance_Category_Account terms and changes      ...       State_State57  \
0                                             0      ...                   0   
1                                             0      ...                   0   
2                                             0      ...                   0   
3                                             0      ...                   0   
4                                             0      ...                   0   

   State_State58  State_State59  State_State6  State_State60  State_State61  \
0              0              0             0              0              0   
1              0              0             0              0              0   
2              0              0             0              0              0   
3              0              0             0              0              0   
4              0              0             0              0              0   

   State_State62  State_State7  State_State8  State_State9  
0              0             0             0             0  
1              0             0             0             0  
2              0             0             0             0  
3              0             0             0             0  
4              0             0             0             0  

[5 rows x 144 columns]

In [33]:
test_df.columns.tolist()

['BankGrade',
 'Labels',
 'Processing_time',
 'freq_by_Grievance_Category',
 'freq_by_LineOfBusiness',
 'freq_by_State',
 'Disputed_No',
 'Disputed_Yes',
 'Grievance_Category_APR or interest rate',
 'Grievance_Category_Account terms and changes',
 'Grievance_Category_Advertising and marketing',
 'Grievance_Category_Application processing delay',
 'Grievance_Category_Application, originator, mortgage broker',
 'Grievance_Category_Applied for loan/did not receive money',
 'Grievance_Category_Applying for a mortgage',
 'Grievance_Category_Applying for a mortgage or refinancing an existing mortgage',
 'Grievance_Category_Arbitration',
 'Grievance_Category_Balance transfer',
 'Grievance_Category_Balance transfer fee',
 'Grievance_Category_Bankruptcy',
 'Grievance_Category_Billing disputes',
 'Grievance_Category_Billing statement',
 "Grievance_Category_Can't contact lender",
 "Grievance_Category_Can't repay my loan",
 "Grievance_Category_Can't stop charges to bank account",
 'Grievance_Categ

In [34]:
test_df = test_df.fillna(0)

In [37]:
test_df = test_df.drop(['BankGrade'],1)

In [42]:
train_df.BankGrade = train_df.BankGrade.astype('int')
train_df.head()

BankGrade  Labels  Processing_time  freq_by_Grievance_Category  \
0          2       1               11                    0.021069   
1          1       1               11                    0.021069   
2          1       1               22                    0.021069   
3          2       1               16                    0.021069   
4          1       1               11                    0.021069   

   freq_by_LineOfBusiness  freq_by_State  Disputed_No  Disputed_Yes  \
0                  0.3962       0.012593            1             0   
1                  0.3962       0.012593            1             0   
2                  0.3962       0.012593            0             1   
3                  0.3962       0.012593            1             0   
4                  0.3962       0.012593            1             0   

   Grievance_Category_APR or interest rate  \
0                                        0   
1                                        0   
2                                        0   
3                                        0   
4                                        0   

   Grievance_Category_Account terms and changes      ...       State_State57  \
0                                             0      ...                   0   
1                                             0      ...                   0   
2                                             0      ...                   0   
3                                             0      ...                   0   
4                                             0      ...                   0   

   State_State58  State_State59  State_State6  State_State60  State_State61  \
0              0              0             0              0              0   
1              0              0             0              0              0   
2              0              0             0              0              0   
3              0              0             0              0              0   
4              0              0             0              0              0   

   State_State62  State_State7  State_State8  State_State9  
0              0             0             0             0  
1              0             0             0             0  
2              0             0             0             0  
3              0             0             0             0  
4              0             0             0             0  

[5 rows x 144 columns]

In [43]:
test_df.head()

Labels  Processing_time  freq_by_Grievance_Category  \
0       0               11                         0.0   
1       0               15                         0.0   
2       1               11                         0.0   
3       1               11                         0.0   
4       0               11                         0.0   

   freq_by_LineOfBusiness  freq_by_State  Disputed_No  Disputed_Yes  \
0                     0.0            0.0            1             0   
1                     0.0            0.0            1             0   
2                     0.0            0.0            1             0   
3                     0.0            0.0            1             0   
4                     0.0            0.0            1             0   

   Grievance_Category_APR or interest rate  \
0                                        0   
1                                        0   
2                                        0   
3                                        0   
4                                        0   

   Grievance_Category_Account terms and changes  \
0                                             0   
1                                             0   
2                                             0   
3                                             0   
4                                             0   

   Grievance_Category_Advertising and marketing      ...       State_State57  \
0                                             0      ...                   0   
1                                             0      ...                   0   
2                                             0      ...                   0   
3                                             0      ...                   0   
4                                             0      ...                   0   

   State_State58  State_State59  State_State6  State_State60  State_State61  \
0              0              0             0              0              0   
1              0              0             0              0              0   
2              0              0             0              0              0   
3              0              0             0              0              0   
4              0              0             0              0              0   

   State_State62  State_State7  State_State8  State_State9  
0              0             0             0             0  
1              0             0             0             0  
2              0             0             0             0  
3              0             0             0             0  
4              0             0             0             0  

[5 rows x 143 columns]

## Model Building 

##### Processed data combined with dummification will undergo train test split 

In [38]:
# Sklearn package sets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *

# Machine learning

import xgboost as xgb

# Dealing with datetime features
from datetime import *


# Timer
import time

### Set seed

In [39]:
# global seed
rs = 20171101
np.random.seed(rs)

** 2. Split training, validation and test set **<br>
- training set: 70%
- validation set: 30%


In [44]:
train_len = int(train_df.shape[0] * 0.7)
train_len

37576

In [46]:
train = train_df.iloc[:train_len , :]
test = train_df.iloc[train_len: , :]

print('Training set shape:', train.shape)
print('Test set shape:', test.shape)

Training set shape: (37576, 144)
Test set shape: (16104, 144)


#### Separating the labels

In [47]:
Label = train_df.BankGrade

In [50]:
del train_df['BankGrade']

In [52]:

X_train, X_val, y_train, y_val = train_test_split(
    train_df, Label, 
    test_size = 0.3, 
    random_state = rs)

In [53]:
print('X_train shape:', X_train.shape)
print('X_val shape:', X_val.shape)
print('y_train shape:', y_train.shape)
print('y_val shape:', y_val.shape)

X_train shape: (37576, 143)
X_val shape: (16104, 143)
y_train shape: (37576,)
y_val shape: (16104,)


In [54]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

### Model using MultinomialNB

In [55]:
mdbmodel = MultinomialNB()
mdbmodel.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [56]:
mnb_predictions = mdbmodel.predict(X_val)

In [57]:
print(accuracy_score(y_pred=mnb_predictions, y_true=y_val))

0.6282911077993045


In [58]:
precision = precision_score(y_pred=mnb_predictions, y_true=y_val, average='weighted')
print(precision)
recall = recall_score(y_pred=mnb_predictions, y_true=y_val, average='weighted')
print(recall)

0.647404120035191
0.6282911077993045


In [59]:
mnb_predictions_on_test = mdbmodel.predict(test_df)

In [68]:
new_df['Predictions_MNB'] = mdbmodel.predict(test_df)

### Random Forest Classifier

In [64]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [66]:
rf.fit(X_train, y_train)
predicted_rf = rf.predict(X_val)
print('Accuray of the model is ', accuracy_score(y_val, predicted_rf))

Accuray of the model is  0.648224043715847


In [67]:
new_df['Predictions'] = rf.predict(test_df)

## Xgboost 

In [69]:
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier()

In [70]:
trained_model=xgb.fit(X_train,y_train)
print(trained_model)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)


In [71]:
y_pred = xgb.predict(X_val)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [72]:
predictions_xgb = [round(value) for value in y_pred]

In [74]:
accuracy = accuracy_score(y_val, predictions_xgb)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 66.91%


In [75]:
new_df['Prediction_xgb'] = xgb.predict(test_df)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## SVM

In [76]:
svmmodel = SVC(C=20, gamma=0.2)
svmmodel.fit(X=X_train, y=y_train)

SVC(C=20, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.2, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [77]:
svm_predictions = svmmodel.predict(X_val)

In [78]:
print('Accuracy-',accuracy_score(y_pred=svm_predictions, y_true=y_val))

Accuracy- 0.6544957774465971


In [79]:
new_df['Prediction_svm'] = svmmodel.predict(test_df)

In [80]:
new_df.to_csv("results.csv")

In [82]:
new_dataframe  = pd.merge(new_df , new_data , how='left')

In [83]:
new_dataframe.to_csv('new_results.csv')

## The file new_df will contain the result of all four models . 


## Additional approaches like Ensemble Learning , with average weighted approach can be performed on the outputs of the four models to give better accuracy 